pip installs

In [1]:
#pip install requests

imports

In [76]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from sklearn.preprocessing import StandardScaler, LabelEncoder
# from sklearn.model_selection import train_test_split 
# from sklearn import datasets, tree
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score, recall_score, precision_score
%matplotlib inline
import plotly.graph_objects as go
import plotly.io as pio  # me permite dibujar en un notebook
pio.renderers.default = "notebook"  # me permite dibujar en un notebook
import plotly.express as px
import plotly.figure_factory as ff
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.model_selection import GridSearchCV
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import itertools
import requests
import os
from collections import Counter

load/write labels and parameters

In [77]:

data_start_label='Parameter'
Parameter_data_rows = ['Unit','HighL','LowL','Tests#','Patterns']
HardBinDictName='HardBinName'
SoftBinDictName='SoftBinName'
header_rows_drop = 4
datatypes_per_unit = {
    'VOLTS':'float64',
    'Ohms':'float64',
    'V':'float64',
    'uSECONDS':'float64',
    'mAMPS':'float64',
    ' ':'float64',
    'code':'int64',
    'C':'float64',
    'LSB':'float64',
    'deg':'float64',
    'PF':'float64',
    'KHERTZ':'float64',
    'MHERTZ':'float64',
    'KkHz':'float64',
    '%':'float64',
    'uAMPS':'float64',
    'nnA':'float64',
    'nAMPS':'float64'}
testdatacols=[' SBIN','HBIN','DIE_X','DIE_Y','SITE','TIME','TOTAL_TESTS','LOT_ID','WAFER_ID']
confdict={'data_start_label':data_start_label,'Parameter_data_rows':Parameter_data_rows,'testdatacols':testdatacols,'HardBinDictName':HardBinDictName,'SoftBinDictName':SoftBinDictName,'datatypes_per_unit':datatypes_per_unit}



Load csv
generate df with data
generate HardBinDict
generate SoftBinDict
generate general_data

In [115]:
def load_data (_file_location, _confdict):
    _rawdf = pd.read_csv(_file_location)  # carga csv
    #future adding exceptions for cases like no string match as "Parameter" to handle and track code breakdown
    _data_start_col_value=(_rawdf.iloc[:,0]==_confdict.get('data_start_label')).idxmax()+1  # search for label in table data start
    _df = _rawdf.drop(_rawdf.index[:_data_start_col_value])
    _df.columns=_rawdf.iloc[_data_start_col_value-1]
    _df.set_index(data_start_label,inplace=True)
    _df.columns.name='Tests'
    # _df = pd.read_csv(_file_location, header=_data_start_col_value, index_col=_confdict.get('data_start_label'))  # load table data from csv into df
    _df.drop(columns=_df.columns[-1], axis=1, inplace=True) #extra column without data

    #separate testdata and parameter data (might get input to make this optional)
    _dftestdata = _df[_confdict.get('testdatacols')].drop(_confdict.get('Parameter_data_rows')).astype({' SBIN':'int','HBIN':'int'})
    _df.drop(columns=_confdict.get('testdatacols'), inplace=True)
    _dfpardata=_df.loc[_confdict.get('Parameter_data_rows')]
    _df.drop(_confdict.get('Parameter_data_rows'), inplace=True)
    _df.reset_index(drop=True)
    #_dfpardata.loc['Unit'].replace(confdict.get('datatypes_per_unit'))

    # 
    _rawdf = _rawdf.head(_data_start_col_value-1)  # drop table data
    _rawdf = _rawdf.tail(-header_rows_drop)  # drop 4 rows from the header
    _HardBinDict = (_rawdf[_rawdf.iloc[:,0]==_confdict.get('HardBinDictName')].iloc[:,[1,2]])
    _HardBinDict = _HardBinDict.set_index(_HardBinDict.iloc[:, 0]).iloc[:,1]
    _HardBinDict = _HardBinDict.to_dict()
    _SoftBinDict = (_rawdf[_rawdf.iloc[:,0]==_confdict.get('SoftBinDictName')].iloc[:,[1,2]])
    _SoftBinDict = _SoftBinDict.set_index(_SoftBinDict.iloc[:, 0]).iloc[:,1]
    _SoftBinDict = _SoftBinDict.to_dict()
    _rawdf = _rawdf[(_rawdf.iloc[:,0]!=_confdict.get('HardBinDictName'))&(_rawdf.iloc[:,0]!=_confdict.get('SoftBinDictName'))]  # drop softbin and hardbin
    _rawdf.dropna(how='all')
    _general_data = _rawdf.iloc[:,[0,1]]  # global data extracted by position
    _general_data = _general_data.set_index(_general_data.iloc[:, 0]).iloc[:,1]
    _general_data = _general_data.to_dict()

    # input corrections
    _dfpardata['HSR_SMM ANGLE_D1']['Unit'] = 'deg'  # unit is "code" and should be "deg"
    _dfpardata['HSR_SMM ANGLE_D2']['Unit'] = 'deg'  # unit is "code" and should be "deg"
    _df = _df.replace(r'^\s*$', np.NaN, regex=True)  # if a chip blows, no further testing is done and a space char is placed instead of a value, cannot use np.NaN for int dtypes
    _dfdropped = _df[_df.isna().any(axis=1)].astype('float')
    _df.dropna(inplace=True)  # not reindexed
    _coldtypes=_dfpardata.loc['Unit'].replace(_confdict.get('datatypes_per_unit')).fillna('float64').to_dict()  # if no unit is stated defaults to float dtype as there are many non integers
    _df = _df.astype(_coldtypes)

    return (_df, _general_data,_dfpardata,_dftestdata, _HardBinDict, _SoftBinDict, _dfdropped)

In [140]:
def findfaultedmeas (_df, _lims):
    _dffaultedHi = _df.gt(_lims.loc['HighL'], axis=1)
    _dffaultedLo = _df.lt(_lims.loc['LowL'], axis=1)
    s=_dffaultedHi.stack()
    s=s[s].index.tolist()
    faultedvalues= [res + ('Hi',) for res in s]
    s=_dffaultedLo.stack()
    s=s[s].index.tolist()
    faultedvalues= faultedvalues + [res + ('Lo',) for res in s]
    return(faultedvalues)
    

In [116]:
file_location1 = "C:/Users/ggrinberg/AllegroMicro/33022D Notebook - Qual Results/STR6129/RAW_DATA/X_HBM/CSV/1-126/QR6129X1.csv"  #"C:/Users/ggrinberg/Downloads/QR6129X1.csv"
file_location2 = "C:/Users/ggrinberg/AllegroMicro/33022D Notebook - Qual Results/STR6129/RAW_DATA/X_HBM/CSV/1-126/QH6129X2.csv"
file_location3 = "C:/Users/ggrinberg/Downloads/QR6129X3.csv"
file_location4 = "C:/Users/ggrinberg/Downloads/QH6129X4.csv"
(df1, general_data1 ,dfpardata1, dftestdata1, HardBinDict, SoftBinDict, df1dropped) = load_data (file_location1, confdict)
(df2, general_data2 ,dfpardata2, dftestdata2, HardBinDict, SoftBinDict, df2dropped) = load_data (file_location2, confdict)
(df3, general_data3 ,dfpardata3, dftestdata3, HardBinDict, SoftBinDict, df3dropped) = load_data (file_location3, confdict)
(df4, general_data4 ,dfpardata4, dftestdata4, HardBinDict, SoftBinDict, df4dropped) = load_data (file_location4, confdict)

In [141]:
Lims1=dfpardata1.loc[['LowL','HighL']].astype('float')
Lims2=dfpardata2.loc[['LowL','HighL']].astype('float')
Lims3=dfpardata3.loc[['LowL','HighL']].astype('float')
Lims4=dfpardata4.loc[['LowL','HighL']].astype('float')

In [143]:
Lims4

Tests,VCC1 OS Pre_D1,VCC1 OS Pre_D2,PWM OS Pre_D1,PWM OS Pre_D2,CSN OS Pre_D1,CSN OS Pre_D2,SCLK OS Pre_D1,SCLK OS Pre_D2,MOSI OS Pre_D1,MOSI OS Pre_D2,...,MOSI OS Pre_D1,MOSI OS Pre_D2,MISO OS Pre_D1,MISO OS Pre_D2,AU OS Pre_D1,AU OS Pre_D2,BV OS Pre_D1,BV OS Pre_D2,IW OS Pre_D1,IW OS Pre_D2
Parameter,,,,,,,,,,,,,,,,,,,,,
LowL,NaN,NaN,-1.5,-1.5,-1.5,-1.5,-1.5,-1.5,-1.5,-1.5,...,-1.5,-1.5,-1.5,-1.5,-1.5,-1.5,-1.5,-1.5,-1.5,-1.5
HighL,-18.0,-18.0,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,...,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1


In [146]:
findfaultedmeas(df3dropped,Lims3)

[('PID-113', 'Secondary Temp Sensor (25C)_D1', 'Hi'),
 ('PID-113', 'Secondary Temp Sensor (25C)_D2', 'Hi'),
 ('PID-113', 'PWM Leakage Current_D1', 'Hi'),
 ('PID-120', 'Secondary Temp Sensor (25C)_D1', 'Hi'),
 ('PID-120', 'Secondary Temp Sensor (25C)_D2', 'Hi'),
 ('PID-120', 'Icc_Hibernation_D1', 'Hi')]

hay columnas con mediciones repetidas pero valores distintos

In [59]:
#df1.columns
measures = Counter(dfpardata1.columns.to_list())
result = [_key for _key, _value in measures.items () if _value != 1]
print(f"there are {len(result)} repeated measurements, check result contents")

hay 61 mediciones repetidas, chequear con pato


In [55]:
#print(f"blank cells filled with{df3['SENT Trigger Threshold L Code 0_D1']['PID-120']}")  # check how an empty cell is filled  # commented as rows now dropped, info in testdata/HardBin
measures = Counter(dfpardata1.columns.to_list()+dfpardata2.columns.to_list()+dfpardata3.columns.to_list()+dfpardata4.columns.to_list())
result = [_key for _key, _value in measures.items () if _value%4 != 0]
print(f"there are {len(result)} of {len(measures)} measurements which are not in the 4 datasets, most are temperature related, check with pato the columns in result var")

there are 87 of 801 measurements which are not in the 4 datasets, most are temperature related, check with pato 


In [66]:
df1.shape[0]

136

In [69]:
Lims1=dfpardata1.loc[['LowL','HighL']].astype('float')
faulted_by_param = (df1.gt(Lims1.loc['HighL'], axis=1) | df1.lt(Lims1.loc['LowL'], axis=1))
print('mediciones fuera de limites para df1 en %')
print(faulted_by_param.sum()[faulted_by_param.any()].sort_values(ascending=False).head(10)/df1.shape[0]*100)  # secondary temp sensor has ill defined thresholds

mediciones fuera de limites para df1 en %
Tests
Secondary Temp Sensor (25C)_D1    100.000000
Secondary Temp Sensor (25C)_D2    100.000000
Primary Temp Sensor (25C)_D2        4.411765
HSR_SMM_D2                          3.676471
HSR_SMM_D1                          2.941176
dtype: float64


In [71]:
Lims2=dfpardata2.loc[['LowL','HighL']].astype('float')
faulted_by_param = (df2.gt(Lims2.loc['HighL'], axis=1) | df2.lt(Lims2.loc['LowL'], axis=1))
print('mediciones fuera de limites para df2 en %')
print(faulted_by_param.sum()[faulted_by_param.any()].sort_values(ascending=False).head(10)/df1.shape[0]*100) 

mediciones fuera de limites para df2 en %
Tests
IWRevZen_D2    50.735294
dtype: float64


In [70]:
Lims3=dfpardata3.loc[['LowL','HighL']].astype('float')
faulted_by_param = (df3.gt(Lims3.loc['HighL'], axis=1) | df3.lt(Lims3.loc['LowL'], axis=1))
print('mediciones fuera de limites para df3 en %')
print(faulted_by_param.sum()[faulted_by_param.any()].sort_values(ascending=False).head(10)/df1.shape[0]*100)  # secondary temp sensor has ill defined thresholds

mediciones fuera de limites para df3 en %
Tests
Secondary Temp Sensor (25C)_D1    91.176471
Secondary Temp Sensor (25C)_D2    91.176471
HSR_SMM_D2                         3.676471
HSR_SMM_D1                         2.941176
Primary Temp Sensor (25C)_D1       1.470588
Primary Temp Sensor (25C)_D2       1.470588
dtype: float64


In [72]:
Lims4=dfpardata4.loc[['LowL','HighL']].astype('float')
faulted_by_param = (df4.gt(Lims4.loc['HighL'], axis=1) | df4.lt(Lims4.loc['LowL'], axis=1))
print('mediciones fuera de limites para df4 en %')
print(faulted_by_param.sum()[faulted_by_param.any()].sort_values(ascending=False).head(10)/df1.shape[0]*100) 

mediciones fuera de limites para df4 en %
Tests
IWRevZen_D2    47.058824
dtype: float64


In [73]:
dftestdata3.dtypes

Tests
 SBIN           int32
HBIN            int32
DIE_X          object
DIE_Y          object
SITE           object
TIME           object
TOTAL_TESTS    object
LOT_ID         object
WAFER_ID       object
dtype: object

In [102]:
print('mayor faults')
print(dftestdata4['HBIN'].astype('str').replace(HardBinDict).value_counts())
print(f"faulted parts:{dftestdata4[dftestdata4['HBIN']!=1].index.tolist()}")

mayor faults
HBIN
Good    124
PWM       1
SPI       1
Name: count, dtype: int64
faulted parts:['PID-113', 'PID-120']


In [90]:
dftestdata3[dftestdata3['HBIN']!=1].index.tolist()

['PID-113', 'PID-120']

In [113]:
df3dropped.astype('float')

Tests,VCC1 OS Pre_D1,VCC1 OS Pre_D2,PWM OS Pre_D1,PWM OS Pre_D2,CSN OS Pre_D1,CSN OS Pre_D2,SCLK OS Pre_D1,SCLK OS Pre_D2,MOSI OS Pre_D1,MOSI OS Pre_D2,...,MOSI OS Pre_D1,MOSI OS Pre_D2,MISO OS Pre_D1,MISO OS Pre_D2,AU OS Pre_D1,AU OS Pre_D2,BV OS Pre_D1,BV OS Pre_D2,IW OS Pre_D1,IW OS Pre_D2
Parameter,,,,,,,,,,,,,,,,,,,,,
PID-113,-18.55769,-19.82925,-0.554847,-0.556102,-0.523255,-0.530546,-0.523180,-0.530359,-0.523105,-0.530827,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PID-120,-18.57460,-19.80339,-0.558977,-0.560134,-0.533350,-0.535061,-0.533837,-0.535847,-0.533313,-0.536222,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [118]:
Lims3=dfpardata3.loc[['LowL','HighL']].astype('float')
df3faultedHi = df3dropped.gt(Lims3.loc['HighL'], axis=1)
df3faultedHi.sum()[df3faultedHi.any()]

Tests
Secondary Temp Sensor (25C)_D1    2
Secondary Temp Sensor (25C)_D2    2
PWM Leakage Current_D1            1
Icc_Hibernation_D1                1
dtype: int64

In [138]:
Lims4=dfpardata4.loc[['LowL','HighL']].astype('float')
df4faultedLo = df4dropped.lt(Lims4.loc['LowL'], axis=1)
df4faultedLo.sum()[df4faultedLo.any()]

Series([], dtype: int64)

In [135]:
Lims3=dfpardata3.loc[['LowL','HighL']].astype('float')
df3faultedHi = df3dropped.gt(Lims3.loc['HighL'], axis=1)
df3faultedLo = df3dropped.lt(Lims3.loc['LowL'], axis=1)
s=df3faultedHi.stack()
s=s[s].index.tolist()
faultedvalues= [res + ('Hi',) for res in s]
s=df3faultedLo.stack()
s=s[s].index.tolist()
faultedvalues= faultedvalues + [res + ('Lo',) for res in s]
faultedvalues

[('PID-113', 'Secondary Temp Sensor (25C)_D1', 'Hi'),
 ('PID-113', 'Secondary Temp Sensor (25C)_D2', 'Hi'),
 ('PID-113', 'PWM Leakage Current_D1', 'Hi'),
 ('PID-120', 'Secondary Temp Sensor (25C)_D1', 'Hi'),
 ('PID-120', 'Secondary Temp Sensor (25C)_D2', 'Hi'),
 ('PID-120', 'Icc_Hibernation_D1', 'Hi')]

In [137]:
Lims4=dfpardata4.loc[['LowL','HighL']].astype('float')
df4faultedHi = df4dropped.gt(Lims4.loc['HighL'], axis=1)
df4faultedLo = df4dropped.lt(Lims4.loc['LowL'], axis=1)
s=df4faultedHi.stack()
s=s[s].index.tolist()
faultedvalues= [res + ('Hi',) for res in s]
s=df4faultedLo.stack()
s=s[s].index.tolist()
faultedvalues= faultedvalues + [res + ('Lo',) for res in s]
faultedvalues

[('PID-113', 'PWM Leakage Current_D1', 'Hi'),
 ('PID-120', 'Icc_Hibernation_D1', 'Hi')]

In [133]:
s=df3faultedHi.stack()
s=s[s].index.tolist()
faultedvalues= [res + ('Hi',) for res in s]
faultedvalues

[('PID-113', 'Secondary Temp Sensor (25C)_D1', 'Hi'),
 ('PID-113', 'Secondary Temp Sensor (25C)_D2', 'Hi'),
 ('PID-113', 'PWM Leakage Current_D1', 'Hi'),
 ('PID-120', 'Secondary Temp Sensor (25C)_D1', 'Hi'),
 ('PID-120', 'Secondary Temp Sensor (25C)_D2', 'Hi'),
 ('PID-120', 'Icc_Hibernation_D1', 'Hi')]

In [131]:
s

Parameter  Tests         
PID-113    VCC1 OS Pre_D1    False
           VCC1 OS Pre_D2    False
           PWM OS Pre_D1     False
           PWM OS Pre_D2     False
           CSN OS Pre_D1     False
                             ...  
PID-120    AU OS Pre_D2      False
           BV OS Pre_D1      False
           BV OS Pre_D2      False
           IW OS Pre_D1      False
           IW OS Pre_D2      False
Length: 1624, dtype: bool

In [87]:
for index, row in dftestdata3.iterrows():
    if row['HBIN']!=1:
        print(row['HBIN'])

13
25
